<a href="https://colab.research.google.com/github/Anidafio/4105_hw4/blob/main/hw5_p3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/Anidafio/4105_hw5/main/Housing.csv'

housing = pd.read_csv(url)
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [ ]:
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

housing[varlist] = housing[varlist].apply(binary_map)

variables =  ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'parking']

X = housing[variables].copy().values
Y = housing[['price']].copy().values

from sklearn.preprocessing import MinMaxScaler
nc_X = MinMaxScaler()
X = nc_X.fit_transform(X)
Y = nc_X.fit_transform(Y)

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 100)

In [ ]:
import torch
import torch.optim as optim

X_train = torch.tensor(X_train)
Y_train = torch.tensor(Y_train)
X_test = torch.tensor(X_test)
Y_test = torch.tensor(Y_test)

In [ ]:
#def model(t_u, w, b):
#  return w*t_u + b

def model(t_u, w11, w10, w9, w8, w7, w6, w5, w4, w3, w2, w1, b):
  return w11*t_u[:,10] + w10*t_u[:,9] + w9*t_u[:,8] + w8*t_u[:,7] + w7*t_u[:,6] + w6*t_u[:,5] + w5*t_u[:,4] + w4*t_u[:,3] + w3*t_u[:,2] + w2*t_u[:,1] + w1*t_u[:,0] + b

def loss_fn(t_p, t_c):
  squared_diffs = (t_p - t_c)**2
  return squared_diffs.mean()

def training_loop(n_epochs, optimizer, params, train_t_u, train_t_c, val_t_u, val_t_c):
  for epoch in range(1, n_epochs+1):
    train_t_p = model(train_t_u, *params)
    train_loss = loss_fn(train_t_p, train_t_c)

    val_t_p = model(val_t_u, *params)
    val_loss = loss_fn(val_t_p, val_t_c)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch%500 == 0:
      print('Epoch %d, Training loss %f, validation loss %f, ' % (epoch, float(train_loss), float(val_loss)))

  return params

In [ ]:
params = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
optimizer = optim.Adam([params], lr = 0.1)

training_loop(
    n_epochs=5000,
    optimizer=optimizer,
    params=params,
    train_t_u = X_train,
    train_t_c = Y_train,
    val_t_u = X_test,
    val_t_c = Y_test
)

Epoch 500, Training loss 0.026082, validation loss 0.026631, 
Epoch 1000, Training loss 0.026076, validation loss 0.026615, 
Epoch 1500, Training loss 0.026076, validation loss 0.026615, 
Epoch 2000, Training loss 0.026076, validation loss 0.026615, 
Epoch 2500, Training loss 0.026076, validation loss 0.026615, 
Epoch 3000, Training loss 0.026076, validation loss 0.026615, 
Epoch 3500, Training loss 0.026076, validation loss 0.026615, 
Epoch 4000, Training loss 0.026076, validation loss 0.026615, 
Epoch 4500, Training loss 0.026076, validation loss 0.026615, 
Epoch 5000, Training loss 0.026076, validation loss 0.026615, 


tensor([ 9.1149e-11,  6.1721e-11, -6.4110e-11, -7.6302e-11, -9.7746e-11,
         5.4415e-11, -9.8039e-10,  1.9207e-10,  4.9975e-10, -2.8590e-09,
        -6.7233e-10,  2.6370e-01], requires_grad=True)